In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn import ensemble



In [2]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV

In [3]:
from sklearn.feature_selection import SelectKBest, f_regression

## DRILL: Third Attempt

So here's your task. Get rid of as much data as possible without dropping below an average of 90% accuracy in a 10-fold cross validation.

You'll want to do a few things in this process. First, dive into the data that we have and see which features are most important. This can be the raw features or the generated dummies. You may want to use PCA or correlation matrices.

Can you do it without using anything related to payment amount or outstanding principal? How do you know?

In [4]:
df = pd.read_csv('https://www.dropbox.com/s/0so14yudedjmm5m/LoanStats3d.csv?dl=1',
                 skipinitialspace=True, header=1)
df = df[:-2]
#df = pd.read_csv('data/y2015.csv', header=0)
#df = df.drop(df.columns[0], axis=1)  

C:\Users\chuck\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
drop0 = [label for label in df.columns if len(df[label].unique()) >= 100]
drop1 = ['loan_status', 'grade', 'pymnt_plan', 'desc', 'addr_state', 'initial_list_status', 'next_pymnt_d', 'verification_status_joint'] 
data = df.drop(drop0, axis=1)
data = data.drop(drop1, axis=1)

In [6]:
col_nan = data.columns[np.where(data.isna().any())]
col_flt = data.columns[np.where(data.dtypes == 'float64')]
col_obt = data.columns[np.where(data.dtypes == 'object')]
col_not = list(set(col_nan) & set(col_obt))
col_fnn = list(set(col_flt) & set(col_nan))
col_ooy = list(set(col_obt) - set(col_nan))
col_foy = list(set(col_flt) - set(col_nan))

In [7]:
numeric_features = col_flt
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = col_obt
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

select = SelectKBest(f_regression, k=30)

rfc = ensemble.RandomForestClassifier()

# feature selector

# pipeline appending classifier
clf = Pipeline(steps=[('pre', preprocessor),  
                      ('slt', select),
                      ('rfc', rfc)])

In [8]:
X = data
y = df['loan_status']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [9]:
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(clf.score(X_test, y_test))

TypeError: unsupported operand type(s) for /: 'str' and 'int'